In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble, model_selection, metrics, neural_network, preprocessing

In [2]:
X = pd.read_csv('~/data/MLProject/xTrainFinal.csv')
Y = pd.read_csv('~/data/MLProject/y_train.csv')

In [45]:
columns = [col for col in X.columns if "std" not in col and "med" not in col]
X = X[columns]
X['cluster'] = X['sample_id'] // 21
X['day'] = X['sample_id'] % 21
X

,sample_id,md,bc,ret_1,ret_2,ret_3,ret_4,ret_5,ret_6,ret_7,...,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23,size_cluster,prop_correlated,cluster,day
0,0,-5.705550,0.599596,-0.003180,0.002505,0.002466,-0.010499,0.001232,-0.004430,-0.006318,...,0.006920,0.005116,0.002664,0.009894,0.002198,0.006052,19.0,0.099723,0,0
1,1,-5.705550,0.599596,0.013303,0.001221,0.000866,-0.013438,0.014031,0.007980,-0.005909,...,0.003216,0.005870,-0.021580,-0.011090,0.005877,0.001482,19.0,0.099723,0,1
2,2,-5.705550,0.599596,-0.012115,0.023476,0.010832,0.004019,-0.001082,-0.015029,-0.003969,...,0.009375,0.001003,-0.012442,-0.020219,-0.011961,0.009295,19.0,0.099723,0,2
3,3,-5.705550,0.599596,-0.008271,0.007541,-0.002724,-0.001246,0.000680,0.020786,-0.002515,...,-0.000502,-0.012990,-0.014617,-0.011488,0.004350,0.018875,19.0,0.099723,0,3
4,4,-5.705550,0.599596,0.011887,0.001898,-0.010022,-0.013176,0.003421,0.008634,0.005427,...,0.004123,0.019168,-0.001773,-0.001574,0.000908,0.005251,19.0,0.099723,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30489,30739,-6.751337,0.471494,-0.024104,-0.016507,-0.005597,0.008415,-0.001305,-0.001000,-0.017745,...,-0.003174,0.000514,0.002982,0.013443,0.010602,-0.002606,6.0,0.277778,1463,16
30490,30740,-6.751337,0.471494,-0.007096,-0.012172,-0.001522,-0.007782,0.003664,0.005777,0.001113,...,-0.000665,-0.004928,-0.008155,0.008358,-0.011834,-0.006234,6.0,0.277778,1463,17
30491,30741,-6.751337,0.471494,0.008908,0.003378,0.001637,0.007419,-0.004770,-0.005766,-0.009467,...,0.003667,0.002937,0.027649,0.005071,0.004436,-0.003866,6.0,0.277778,1463,18
30492,30742,-6.751337,0.471494,0.006361,-0.001831,-0.001641,-0.005485,0.018168,0.006787,-0.002720,...,0.001878,-0.000731,0.003665,-0.007465,-0.002912,-0.002791,6.0,0.277778,1463,19


In [47]:
X2 = X.drop(columns=["md", "bc", "size_cluster", "prop_correlated"]).rename(columns={col: col+"_2" for col in columns if col != "cluster" and col != "day"})
X2

,sample_id_2,ret_1_2,ret_2_2,ret_3_2,ret_4_2,ret_5_2,ret_6_2,ret_7_2,ret_8_2,ret_9_2,...,ret_16_2,ret_17_2,ret_18_2,ret_19_2,ret_20_2,ret_21_2,ret_22_2,ret_23_2,cluster,day
0,0,-0.003180,0.002505,0.002466,-0.010499,0.001232,-0.004430,-0.006318,-0.017592,0.002167,...,0.005674,-0.006023,0.006920,0.005116,0.002664,0.009894,0.002198,0.006052,0,0
1,1,0.013303,0.001221,0.000866,-0.013438,0.014031,0.007980,-0.005909,0.003051,0.001049,...,0.013930,0.005944,0.003216,0.005870,-0.021580,-0.011090,0.005877,0.001482,0,1
2,2,-0.012115,0.023476,0.010832,0.004019,-0.001082,-0.015029,-0.003969,0.007315,0.011428,...,-0.012665,-0.007998,0.009375,0.001003,-0.012442,-0.020219,-0.011961,0.009295,0,2
3,3,-0.008271,0.007541,-0.002724,-0.001246,0.000680,0.020786,-0.002515,-0.015067,0.004542,...,0.012996,0.006508,-0.000502,-0.012990,-0.014617,-0.011488,0.004350,0.018875,0,3
4,4,0.011887,0.001898,-0.010022,-0.013176,0.003421,0.008634,0.005427,0.001078,0.000837,...,0.004875,-0.005173,0.004123,0.019168,-0.001773,-0.001574,0.000908,0.005251,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30489,30739,-0.024104,-0.016507,-0.005597,0.008415,-0.001305,-0.001000,-0.017745,-0.016250,0.012205,...,-0.001303,0.000982,-0.003174,0.000514,0.002982,0.013443,0.010602,-0.002606,1463,16
30490,30740,-0.007096,-0.012172,-0.001522,-0.007782,0.003664,0.005777,0.001113,0.003364,-0.000147,...,0.006559,0.001790,-0.000665,-0.004928,-0.008155,0.008358,-0.011834,-0.006234,1463,17
30491,30741,0.008908,0.003378,0.001637,0.007419,-0.004770,-0.005766,-0.009467,-0.002242,0.001368,...,-0.001815,-0.001511,0.003667,0.002937,0.027649,0.005071,0.004436,-0.003866,1463,18
30492,30742,0.006361,-0.001831,-0.001641,-0.005485,0.018168,0.006787,-0.002720,0.002645,-0.012845,...,-0.002254,0.000997,0.001878,-0.000731,0.003665,-0.007465,-0.002912,-0.002791,1463,19


In [53]:
X1 = X.merge(Y)
X1

,sample_id,md,bc,ret_1,ret_2,ret_3,ret_4,ret_5,ret_6,ret_7,...,ret_19,ret_20,ret_21,ret_22,ret_23,size_cluster,prop_correlated,cluster,day,target
0,0,-5.705550,0.599596,-0.003180,0.002505,0.002466,-0.010499,0.001232,-0.004430,-0.006318,...,0.005116,0.002664,0.009894,0.002198,0.006052,19.0,0.099723,0,0,0.000301
1,1,-5.705550,0.599596,0.013303,0.001221,0.000866,-0.013438,0.014031,0.007980,-0.005909,...,0.005870,-0.021580,-0.011090,0.005877,0.001482,19.0,0.099723,0,1,-0.011966
2,2,-5.705550,0.599596,-0.012115,0.023476,0.010832,0.004019,-0.001082,-0.015029,-0.003969,...,0.001003,-0.012442,-0.020219,-0.011961,0.009295,19.0,0.099723,0,2,-0.011395
3,3,-5.705550,0.599596,-0.008271,0.007541,-0.002724,-0.001246,0.000680,0.020786,-0.002515,...,-0.012990,-0.014617,-0.011488,0.004350,0.018875,19.0,0.099723,0,3,0.000664
4,4,-5.705550,0.599596,0.011887,0.001898,-0.010022,-0.013176,0.003421,0.008634,0.005427,...,0.019168,-0.001773,-0.001574,0.000908,0.005251,19.0,0.099723,0,4,0.002852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30489,30739,-6.751337,0.471494,-0.024104,-0.016507,-0.005597,0.008415,-0.001305,-0.001000,-0.017745,...,0.000514,0.002982,0.013443,0.010602,-0.002606,6.0,0.277778,1463,16,0.014456
30490,30740,-6.751337,0.471494,-0.007096,-0.012172,-0.001522,-0.007782,0.003664,0.005777,0.001113,...,-0.004928,-0.008155,0.008358,-0.011834,-0.006234,6.0,0.277778,1463,17,0.001465
30491,30741,-6.751337,0.471494,0.008908,0.003378,0.001637,0.007419,-0.004770,-0.005766,-0.009467,...,0.002937,0.027649,0.005071,0.004436,-0.003866,6.0,0.277778,1463,18,-0.001114
30492,30742,-6.751337,0.471494,0.006361,-0.001831,-0.001641,-0.005485,0.018168,0.006787,-0.002720,...,-0.000731,0.003665,-0.007465,-0.002912,-0.002791,6.0,0.277778,1463,19,-0.001417


In [54]:
X1['day'] += 1
X1 = X1.rename(columns={'target': 'ret_24'})
X1

,sample_id,md,bc,ret_1,ret_2,ret_3,ret_4,ret_5,ret_6,ret_7,...,ret_19,ret_20,ret_21,ret_22,ret_23,size_cluster,prop_correlated,cluster,day,ret_24
0,0,-5.705550,0.599596,-0.003180,0.002505,0.002466,-0.010499,0.001232,-0.004430,-0.006318,...,0.005116,0.002664,0.009894,0.002198,0.006052,19.0,0.099723,0,1,0.000301
1,1,-5.705550,0.599596,0.013303,0.001221,0.000866,-0.013438,0.014031,0.007980,-0.005909,...,0.005870,-0.021580,-0.011090,0.005877,0.001482,19.0,0.099723,0,2,-0.011966
2,2,-5.705550,0.599596,-0.012115,0.023476,0.010832,0.004019,-0.001082,-0.015029,-0.003969,...,0.001003,-0.012442,-0.020219,-0.011961,0.009295,19.0,0.099723,0,3,-0.011395
3,3,-5.705550,0.599596,-0.008271,0.007541,-0.002724,-0.001246,0.000680,0.020786,-0.002515,...,-0.012990,-0.014617,-0.011488,0.004350,0.018875,19.0,0.099723,0,4,0.000664
4,4,-5.705550,0.599596,0.011887,0.001898,-0.010022,-0.013176,0.003421,0.008634,0.005427,...,0.019168,-0.001773,-0.001574,0.000908,0.005251,19.0,0.099723,0,5,0.002852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30489,30739,-6.751337,0.471494,-0.024104,-0.016507,-0.005597,0.008415,-0.001305,-0.001000,-0.017745,...,0.000514,0.002982,0.013443,0.010602,-0.002606,6.0,0.277778,1463,17,0.014456
30490,30740,-6.751337,0.471494,-0.007096,-0.012172,-0.001522,-0.007782,0.003664,0.005777,0.001113,...,-0.004928,-0.008155,0.008358,-0.011834,-0.006234,6.0,0.277778,1463,18,0.001465
30491,30741,-6.751337,0.471494,0.008908,0.003378,0.001637,0.007419,-0.004770,-0.005766,-0.009467,...,0.002937,0.027649,0.005071,0.004436,-0.003866,6.0,0.277778,1463,19,-0.001114
30492,30742,-6.751337,0.471494,0.006361,-0.001831,-0.001641,-0.005485,0.018168,0.006787,-0.002720,...,-0.000731,0.003665,-0.007465,-0.002912,-0.002791,6.0,0.277778,1463,20,-0.001417


In [59]:
D2 = X1.merge(X2)
D2 = D2.drop(columns=['sample_id', 'cluster', 'day']).rename(columns={'sample_id_2': 'sample_id'})
dataSet = D2.merge(Y)

In [62]:
A = dataSet.drop(columns='target').set_index('sample_id')
B = dataSet[['sample_id', 'target']].set_index('sample_id')

In [74]:
# xTrain, xTest, yTrain, yTest = model_selection.train_test_split(X.set_index('sample_id'), Y.set_index('sample_id'), test_size=.2)
xTrain, xTest, yTrain, yTest = model_selection.train_test_split(A, B, test_size=.2)
reg = ensemble.HistGradientBoostingRegressor(verbose=2, max_leaf_nodes=None, max_iter=200, min_samples_leaf=10, random_state=1000, validation_fraction=.1, tol=1e-9)

In [ ]:
reg.fit(xTrain, yTrain)

In [76]:
print(int(np.sqrt(metrics.mean_squared_error(yTest, reg.predict(xTest))) * 10000))

71


In [81]:
# dataX, dataY = X.set_index('sample_id'), Y.set_index('sample_id')
dataX, dataY = A, B
normalizerX = preprocessing.MinMaxScaler()
normalizerY = preprocessing.MinMaxScaler()
dataX = normalizerX.fit_transform(dataX)
dataY = normalizerY.fit_transform(dataY)
xTrain, xTest, yTrain, yTest = model_selection.train_test_split(dataX, dataY, test_size=.2)
reg = neural_network.MLPRegressor(verbose=True, hidden_layer_sizes=(200, 100, 10), tol=1e-6, learning_rate_init=5e-4)

In [82]:
reg.fit(xTrain, yTrain)

/home/lwi/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.01138714
Iteration 2, loss = 0.00184115
Iteration 3, loss = 0.00169298
Iteration 4, loss = 0.00159003
Iteration 5, loss = 0.00150697
Iteration 6, loss = 0.00143300
Iteration 7, loss = 0.00138915
Iteration 8, loss = 0.00134530
Iteration 9, loss = 0.00131077
Iteration 10, loss = 0.00128757
Iteration 11, loss = 0.00127413
Iteration 12, loss = 0.00126058
Iteration 13, loss = 0.00123142
Iteration 14, loss = 0.00123064
Iteration 15, loss = 0.00122402
Iteration 16, loss = 0.00119410
Iteration 17, loss = 0.00118566
Iteration 18, loss = 0.00118295
Iteration 19, loss = 0.00116852
Iteration 20, loss = 0.00119377
Iteration 21, loss = 0.00115274
Iteration 22, loss = 0.00114496
Iteration 23, loss = 0.00114486
Iteration 24, loss = 0.00115216
Iteration 25, loss = 0.00112525
Iteration 26, loss = 0.00116181
Iteration 27, loss = 0.00113891
Iteration 28, loss = 0.00111196
Iteration 29, loss = 0.00113573
Iteration 30, loss = 0.00111872
Iteration 31, loss = 0.00108678
Iteration 32, los

MLPRegressor(hidden_layer_sizes=(75, 50, 10), learning_rate_init=0.0005,
             tol=1e-05, verbose=True)

In [83]:
pred = reg.predict(xTest)
pred = normalizerY.inverse_transform(pred.reshape(-1, 1))
yTest = normalizerY.inverse_transform(yTest)
print(int(np.sqrt(metrics.mean_squared_error(yTest, pred)) * 10000))

94
